In [1]:
# Import necessary libraries for data manipulation, machine learning, and interface building
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings
import pickle
import gradio as gr

# Ignore warnings to keep the output clean
warnings.filterwarnings('ignore')

c:\Users\Varun\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_excel('../data/satisfaction_2015.xlsx')
df.to_csv('../data/direct.csv', index=False)

In [4]:
df = pd.read_csv('../data/direct.csv')

In [5]:
df.columns = df.columns.str.replace(' ', '_').str.upper()

df.rename(columns={'SATISFACTION_V2': 'SATISFACTION'}, inplace=True)

columns = list(df.columns)
columns.remove('SATISFACTION')
columns.append('SATISFACTION')
df = df[columns]


In [6]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in '{column}': {unique_values}\n")

# Update 'CUSTOMER_TYPE'
df['CUSTOMER_TYPE'] = df['CUSTOMER_TYPE'].replace({
    'disloyal Customer': 'Disloyal',
    'Loyal Customer': 'Loyal'
})

# Update 'TYPE_OF_TRAVEL'
df['TYPE_OF_TRAVEL'] = df['TYPE_OF_TRAVEL'].replace({
    'Personal Travel': 'Personal',
    'Business travel': 'Business'
})

# Update 'SATISFACTION'
df['SATISFACTION'] = df['SATISFACTION'].replace({
    'satisfied': 'Satisfied',
    'neutral or dissatisfied': 'Neutral or Dissatisfied'
})

Unique values in 'ID': [117135  72091  29663 ...  50316  22333  10198]

Unique values in 'GENDER': ['Male' 'Female']

Unique values in 'CUSTOMER_TYPE': ['disloyal Customer' 'Loyal Customer']

Unique values in 'AGE': [56 49 55 36 15 51 26 37 65 18 22 40 14 45 53 35 57 25 10 60 17 68 28 47
 52 34 13 50 64 39 70  8 32 21 67 62 69 63 12 59 27 20 61 30 66 46 54 38
 11  7 24 33  9 23 31 58 19 42 16 29 43 48 44 41 72 71 80 77 85 78 75 79
 74 73 76]

Unique values in 'TYPE_OF_TRAVEL': ['Personal Travel' 'Business travel']

Unique values in 'CLASS': ['Eco' 'Business' 'Eco Plus']

Unique values in 'FLIGHT_DISTANCE': [ 369 2486 1448 ...  681 3763 3564]

Unique values in 'INFLIGHT_WIFI_SERVICE': [0 1 2 3 4 5]

Unique values in 'DEPARTURE/ARRIVAL_TIME_CONVENIENT': [2 3 4 5 0 1]

Unique values in 'EASE_OF_ONLINE_BOOKING': [0 1 3 2 5 4]

Unique values in 'GATE_LOCATION': [4 3 2 5 1 0]

Unique values in 'FOOD_AND_DRINK': [3 2 4 1 5 0]

Unique values in 'ONLINE_BOARDING': [0 1 5 3 2 4]

Unique values i

In [7]:
df.columns

Index(['ID', 'GENDER', 'CUSTOMER_TYPE', 'AGE', 'TYPE_OF_TRAVEL', 'CLASS',
       'FLIGHT_DISTANCE', 'INFLIGHT_WIFI_SERVICE',
       'DEPARTURE/ARRIVAL_TIME_CONVENIENT', 'EASE_OF_ONLINE_BOOKING',
       'GATE_LOCATION', 'FOOD_AND_DRINK', 'ONLINE_BOARDING', 'SEAT_COMFORT',
       'INFLIGHT_ENTERTAINMENT', 'ON-BOARD_SERVICE', 'LEG_ROOM_SERVICE',
       'BAGGAGE_HANDLING', 'CHECKIN_SERVICE', 'INFLIGHT_SERVICE',
       'CLEANLINESS', 'DEPARTURE_DELAY_IN_MINUTES', 'ARRIVAL_DELAY_IN_MINUTES',
       'SATISFACTION'],
      dtype='object')

In [8]:
df.head()

,ID,GENDER,CUSTOMER_TYPE,AGE,TYPE_OF_TRAVEL,CLASS,FLIGHT_DISTANCE,INFLIGHT_WIFI_SERVICE,DEPARTURE/ARRIVAL_TIME_CONVENIENT,EASE_OF_ONLINE_BOOKING,...,INFLIGHT_ENTERTAINMENT,ON-BOARD_SERVICE,LEG_ROOM_SERVICE,BAGGAGE_HANDLING,CHECKIN_SERVICE,INFLIGHT_SERVICE,CLEANLINESS,DEPARTURE_DELAY_IN_MINUTES,ARRIVAL_DELAY_IN_MINUTES,SATISFACTION
0,117135,Male,Disloyal,56,Personal,Eco,369,0,2,0,...,3,1,5,3,3,4,3,0,0.0,Satisfied
1,72091,Male,Disloyal,49,Personal,Eco,2486,0,2,1,...,2,1,1,4,4,3,2,0,0.0,Satisfied
2,29663,Male,Disloyal,55,Personal,Eco,1448,0,3,0,...,3,3,5,3,2,3,3,0,0.0,Satisfied
3,81849,Female,Disloyal,36,Personal,Eco,1501,0,4,0,...,4,5,4,5,5,5,4,0,0.0,Satisfied
4,83693,Male,Disloyal,55,Personal,Eco,577,0,5,0,...,3,3,4,5,3,4,3,0,0.0,Satisfied


In [9]:
df.isnull().sum()

ID                                     0
GENDER                                 0
CUSTOMER_TYPE                          0
AGE                                    0
TYPE_OF_TRAVEL                         0
CLASS                                  0
FLIGHT_DISTANCE                        0
INFLIGHT_WIFI_SERVICE                  0
DEPARTURE/ARRIVAL_TIME_CONVENIENT      0
EASE_OF_ONLINE_BOOKING                 0
GATE_LOCATION                          0
FOOD_AND_DRINK                         0
ONLINE_BOARDING                        0
SEAT_COMFORT                           0
INFLIGHT_ENTERTAINMENT                 0
ON-BOARD_SERVICE                       0
LEG_ROOM_SERVICE                       0
BAGGAGE_HANDLING                       0
CHECKIN_SERVICE                        0
INFLIGHT_SERVICE                       0
CLEANLINESS                            0
DEPARTURE_DELAY_IN_MINUTES             0
ARRIVAL_DELAY_IN_MINUTES             393
SATISFACTION                           0
dtype: int64

In [12]:
df.to_csv('../data/raw.csv', index=False)